# NHANES project: descriptive and regression analysis
> This notebook has the purpose to collect all the analysis on Nhanes dataset for a medical paper project 

Requirements and Information:
1. Nhanes dataset from 1999/00 to 2001/02
2. Oral Frailty Index with:
    - 1: Do you have any difficulties eating tough foods compared to 6 months ago? (OHQ080)
    - 2: Have you choked on your tea or soup recently? (OHQ100 and OHQ105)
    - 3: Do you use dentures? (OHXEDEN)
    - 4: Do you often have a dry mouth?	(OHQ110)
    - 5: Do you go out less frequently than you did last year? (PAQ500)
    - 6: Can you eat hard foods like squid jerky or pickled radish?	(OHQ020)
    - 7: How many times do you brush your teeth in a day? (3 or more times/day)	(OHQ040 and OHQ010)
    - 8:  Do you visit a dental clinic at least annually? (OHQ050)
3. Outcome/Exposure is
4. Demographic Data:
    - Gender (RIAGENDR)
    - Age at screening (RIDAGEYR)
    - Race (RIDRETH1)
    - Education	(DMDEDUC2)
    - Poverty income ratio (INDFMPIR)
    - Smoking status (SMQ020)
5. Confounding Variables:
    - Heart failure	(RIDRETH1)  
    - Coronary heart disease (MCQ160b)
    - Stroke (MCQ160c)
    - COPD (MCQ160f)
    - Liver disease	(MCQ160o)
    - Cancer (MCQ500)
    - Diabetes (MCQ220)
    - High blood pressure (DIQ010)
6. Age => 60 
7. Oral Frailty Index cutoff: 
8. Oral Frailty Index groups:

## Install packages

Reference for nhanesA [here](https://cran.r-project.org/web/packages/nhanesA/nhanesA.pdf)

In [ ]:
install.packages(c("nhanesA", "survey", "MASS", "dplyr", "tidyr", "tidyverse"))


In [ ]:
install.packages("devtools")
devtools::install_github("cjendres1/nhanes")


## Load packages

In [ ]:
library(nhanesA)
library(survey)
library(MASS)
library(dplyr)
library(tidyr)
library(tidyverse)

## Load NHANES data (1999/00-2001/02)

In [5]:
# The browser may be directed to a specific year, survey, or table

browseNHANES(
    year = 1999,
    data_group = "DEMO"
)


In [6]:
# Demographics data

demo_99_00 <- nhanes(nh_table = "DEMO")
demo_01_02 <- nhanes(nh_table = "DEMO_B")


In [ ]:
nhanesTableSummary("DEMO", use = "codebook")

In [8]:
# Smoking status

smoking_99_00 <- nhanes(nh_table = "SMQ")
smoking_01_02 <- nhanes(nh_table = "SMQ_B")

In [9]:
# Oral data

oral_99_00 <- nhanes("OHQ")
oral_01_02 <- nhanes("OHQ_B")

In [10]:
# Physical Activity

physical_99_00 <- nhanes(nh_table = "PAQ")
physical_01_02 <- nhanes(nh_table = "PAQ_B")

In [11]:
# Medical Conditions

med_cond_99_00 <- nhanes("MCQ")
med_cond_01_02 <- nhanes("MCQ_B")

In [12]:
# Diabetes

diabetes_99_00 <- nhanes(nh_table = "DIQ")
diabetes_01_02 <- nhanes(nh_table = "DIQ_B")

In [13]:
# High blood pressure

blood_pressure_99_00 <- nhanes(nh_table = "BPQ")
blood_pressure_01_02 <- nhanes(nh_table = "BPQ_B")

In [14]:
# Edentulous

eden_99_00 <- nhanes(nh_table = "OHXDENT")
eden_01_02 <- nhanes(nh_table = "OHXDEN_B")

## Data Preparation

### Features Selection

In [23]:
# Demographics selection: RIAGENDR, RIDAGEYR, RIDRETH1, DMDEDUC2, INDFMPIR

demo_99_00_selected <- demo_99_00 %>%
    select(SEQN, RIAGENDR, RIDAGEYR, RIDRETH1, DMDEDUC2, INDFMPIR)

demo_01_02_selected <- demo_01_02 %>%
    select(SEQN, RIAGENDR, RIDAGEYR, RIDRETH1, DMDEDUC2, INDFMPIR)

In [24]:
# Smoking Status selection: SMQ020

smoking_99_00_selected <- smoking_99_00 %>%
    select(SEQN, SMQ020)

smoking_01_02_selected <- smoking_01_02 %>%
    select(SEQN, SMQ020)

In [25]:
# Medical Conditions selection: MCQ160B, MCQ160C, MCQ160F, MCQ160K, MCQ160L, MCQ220

med_cond_99_00_selected <- med_cond_99_00 %>%
    select(SEQN, MCQ160B, MCQ160C, MCQ160F, MCQ160K, MCQ160L, MCQ220)

med_cond_01_02_selected <- med_cond_01_02 %>%
    select(SEQN, MCQ160B, MCQ160C, MCQ160F, MCQ160K, MCQ160L, MCQ220)

In [26]:
# Diabetes selection: DIQ010

diabetes_99_00_selected <- diabetes_99_00 %>%
    select(SEQN, DIQ010)

diabetes_01_02_selected <- diabetes_01_02 %>%
    select(SEQN, DIQ010)

In [27]:
# High blood pressure selection: BPQ020

blood_pressure_99_00_selected <- blood_pressure_99_00 %>%
    select(SEQN, BPQ020)

blood_pressure_01_02_selected <- blood_pressure_01_02 %>%
    select(SEQN, BPQ020)

In [28]:
# Physical Activity selection: PAQ500

physical_99_00_selected <- physical_99_00 %>%
    select(SEQN, PAQ500)

physical_01_02_selected <- physical_01_02 %>%
    select(SEQN, PAQ500)

In [29]:
# Edentulous selection: OHXEDEN

eden_99_00_selected <- eden_99_00 %>%
    select(SEQN, OHXEDEN)

eden_01_02_selected <- eden_01_02 %>%
    select(SEQN, OHXEDEN)

In [30]:
# Oral data selection: OHQ080/OHQ085, OHQ100/OHQ105, OHQ110/OHQ115, OHQ020, OHQ040/OHQ010, OHQ050

oral_99_00_selected <- oral_99_00 %>%
    select(SEQN, OHQ080, OHQ100, OHQ110, OHQ020, OHQ040, OHQ050)

oral_01_02_selected <- oral_01_02 %>%
    select(SEQN, OHQ085, OHQ105, OHQ115, OHQ020, OHQ040, OHQ050)

### Data Information

In [31]:
# Function to show dataset information

print_dataset_info <- function(df, name) {
    cat(sprintf("\nDataset: %s\n", name))
    cat(sprintf("Number of rows: %d\n", nrow(df)))
    cat(sprintf("Number of columns: %d\n", ncol(df)))
    cat(sprintf("Memory usage: %.2f MB\n", object.size(df) / 1024^2))
    cat("\nColumn details:\n")
    for (col_name in colnames(df)) {
        cat(sprintf("- %s: %s\n", col_name, class(df[[col_name]])))
    }
}


In [32]:
print_dataset_info(demo_99_00_selected, "Demographics 1999-2000")


Dataset: Demographics 1999-2000
Number of rows: 9965
Number of columns: 6
Memory usage: 0.35 MB

Column details:
- SEQN: numeric
- RIAGENDR: factor
- RIDAGEYR: numeric
- RIDRETH1: factor
- DMDEDUC2: factor
- INDFMPIR: numeric


In [33]:
print_dataset_info(demo_01_02_selected, "Demographics 2001-2002")


Dataset: Demographics 2001-2002
Number of rows: 11039
Number of columns: 6
Memory usage: 0.38 MB

Column details:
- SEQN: numeric
- RIAGENDR: factor
- RIDAGEYR: numeric
- RIDRETH1: factor
- DMDEDUC2: factor
- INDFMPIR: numeric


In [34]:
print_dataset_info(smoking_99_00_selected, "Smoking Status 1999-2000")


Dataset: Smoking Status 1999-2000
Number of rows: 4880
Number of columns: 2
Memory usage: 0.06 MB

Column details:
- SEQN: numeric
- SMQ020: factor


In [35]:
print_dataset_info(smoking_01_02_selected, "Smoking Status 2001-2002")


Dataset: Smoking Status 2001-2002
Number of rows: 5411
Number of columns: 2
Memory usage: 0.06 MB

Column details:
- SEQN: numeric
- SMQ020: factor


In [36]:
print_dataset_info(med_cond_99_00_selected, "Medical Conditions 1999-2000")


Dataset: Medical Conditions 1999-2000
Number of rows: 9493
Number of columns: 7
Memory usage: 0.29 MB

Column details:
- SEQN: numeric
- MCQ160B: factor
- MCQ160C: factor
- MCQ160F: factor
- MCQ160K: factor
- MCQ160L: factor
- MCQ220: factor


In [37]:
print_dataset_info(med_cond_01_02_selected, "Medical Conditions 2001-2002")


Dataset: Medical Conditions 2001-2002
Number of rows: 10470
Number of columns: 7
Memory usage: 0.32 MB

Column details:
- SEQN: numeric
- MCQ160B: factor
- MCQ160C: factor
- MCQ160F: factor
- MCQ160K: factor
- MCQ160L: factor
- MCQ220: factor


In [38]:
print_dataset_info(diabetes_99_00_selected, "Diabetes 1999-2000")


Dataset: Diabetes 1999-2000
Number of rows: 9493
Number of columns: 2
Memory usage: 0.11 MB

Column details:
- SEQN: numeric
- DIQ010: factor


In [39]:
print_dataset_info(diabetes_01_02_selected, "Diabetes 2001-2002")


Dataset: Diabetes 2001-2002
Number of rows: 10470
Number of columns: 2
Memory usage: 0.12 MB

Column details:
- SEQN: numeric
- DIQ010: factor


In [40]:
print_dataset_info(blood_pressure_99_00_selected, "High Blood Pressure 1999-2000")


Dataset: High Blood Pressure 1999-2000
Number of rows: 6044
Number of columns: 2
Memory usage: 0.07 MB

Column details:
- SEQN: numeric
- BPQ020: factor


In [41]:
print_dataset_info(blood_pressure_01_02_selected, "High Blood Pressure 2001-2002")


Dataset: High Blood Pressure 2001-2002
Number of rows: 6634
Number of columns: 2
Memory usage: 0.08 MB

Column details:
- SEQN: numeric
- BPQ020: factor


In [42]:
print_dataset_info(physical_99_00_selected, "Physical Activities 1999-2000")


Dataset: Physical Activities 1999-2000
Number of rows: 9188
Number of columns: 2
Memory usage: 0.11 MB

Column details:
- SEQN: numeric
- PAQ500: factor


In [43]:
print_dataset_info(physical_01_02_selected, "Physical Activities 2001-2002")


Dataset: Physical Activities 2001-2002
Number of rows: 10094
Number of columns: 2
Memory usage: 0.12 MB

Column details:
- SEQN: numeric
- PAQ500: factor


In [44]:
print_dataset_info(eden_99_00_selected, "Edentulous 1999-2000")


Dataset: Edentulous 1999-2000
Number of rows: 8587
Number of columns: 2
Memory usage: 0.10 MB

Column details:
- SEQN: numeric
- OHXEDEN: factor


In [45]:
print_dataset_info(eden_01_02_selected, "Edentulous 2001-2002")


Dataset: Edentulous 2001-2002
Number of rows: 9598
Number of columns: 2
Memory usage: 0.11 MB

Column details:
- SEQN: numeric
- OHXEDEN: factor


In [46]:
print_dataset_info(oral_99_00_selected, "Oral Health 1999-2000")


Dataset: Oral Health 1999-2000
Number of rows: 9230
Number of columns: 7
Memory usage: 0.29 MB

Column details:
- SEQN: numeric
- OHQ080: factor
- OHQ100: factor
- OHQ110: factor
- OHQ020: factor
- OHQ040: factor
- OHQ050: factor


In [47]:
print_dataset_info(oral_01_02_selected, "Oral Health 2001-2002")


Dataset: Oral Health 2001-2002
Number of rows: 10127
Number of columns: 7
Memory usage: 0.31 MB

Column details:
- SEQN: numeric
- OHQ085: factor
- OHQ105: factor
- OHQ115: factor
- OHQ020: factor
- OHQ040: factor
- OHQ050: factor


### Unique values and % for each categorical feature in the datasets

In [48]:
# Function to see unique values for categorical features

print_categorical_info <- function(df, name) {
  cat(sprintf("\nCategorical Column Analysis for Dataset: %s\n", name))

  categorical_cols <- names(df)[sapply(df, is.factor) | sapply(df, is.character)]

  if (length(categorical_cols) == 0) {
    cat("No categorical columns found.\n")
    return()
  }

  for (col_name in categorical_cols) {
    cat(sprintf("\nColumn: %s\n", col_name))
    value_counts <- table(df[[col_name]])  
    total_count <- sum(value_counts)      
    percentages <- round(100 * value_counts / total_count, 2) 

    for (i in seq_along(value_counts)) {
      cat(sprintf("- %s: %d (%.2f%%)\n", names(value_counts)[i], value_counts[i], percentages[i]))
    }
  }
}

In [49]:
print_categorical_info(demo_99_00_selected, "Demographics 1999-2000")


Categorical Column Analysis for Dataset: Demographics 1999-2000

Column: RIAGENDR
- Male: 4883 (49.00%)
- Female: 5082 (51.00%)

Column: RIDRETH1
- Mexican American: 3393 (34.05%)
- Other Hispanic: 589 (5.91%)
- Non-Hispanic White: 3367 (33.79%)
- Non-Hispanic Black: 2228 (22.36%)
- Other Race - Including Multi-Racial: 388 (3.89%)

Column: DMDEDUC2
- Less Than 9th Grade: 932 (19.11%)
- 9-11th Grade (Includes 12th grade with no diploma): 964 (19.76%)
- High School Grad/GED or Equivalent: 1097 (22.49%)
- Some College or AA degree: 1070 (21.94%)
- College Graduate or above: 793 (16.26%)
- Refused: 8 (0.16%)
- Don't Know: 14 (0.29%)


In [50]:
print_categorical_info(demo_01_02_selected, "Demographics 2001-2002")


Categorical Column Analysis for Dataset: Demographics 2001-2002

Column: RIAGENDR
- Male: 5331 (48.29%)
- Female: 5708 (51.71%)

Column: RIDRETH1
- Mexican American: 2776 (25.15%)
- Other Hispanic: 517 (4.68%)
- Non-Hispanic White: 4606 (41.72%)
- Non-Hispanic Black: 2681 (24.29%)
- Other Race - Including Multi-Racial: 459 (4.16%)

Column: DMDEDUC2
- Less Than 9th Grade: 797 (14.74%)
- 9-11th Grade (Includes 12th grade with no diploma): 870 (16.09%)
- High School Grad/GED or Equivalent: 1265 (23.40%)
- Some College or AA degree: 1379 (25.50%)
- College Graduate or above: 1079 (19.96%)
- Refused: 5 (0.09%)
- Don't Know: 12 (0.22%)


In [51]:
print_categorical_info(smoking_99_00_selected, "Smoking Status 1999-2000")


Categorical Column Analysis for Dataset: Smoking Status 1999-2000

Column: SMQ020
- Yes: 2299 (47.14%)
- No: 2566 (52.61%)
- Refused: 4 (0.08%)
- Don't know: 8 (0.16%)


In [52]:
print_categorical_info(smoking_01_02_selected, "Smoking Status 2001-2002")


Categorical Column Analysis for Dataset: Smoking Status 2001-2002

Column: SMQ020
- Yes: 2596 (48.01%)
- No: 2801 (51.80%)
- Refused: 4 (0.07%)
- Don't know: 6 (0.11%)


In [53]:
print_categorical_info(med_cond_99_00_selected, "Medical Conditions 1999-2000")


Categorical Column Analysis for Dataset: Medical Conditions 1999-2000

Column: MCQ160B
- Yes: 167 (3.42%)
- No: 4689 (96.09%)
- Refused: 1 (0.02%)
- Don't know: 23 (0.47%)

Column: MCQ160C
- Yes: 203 (4.16%)
- No: 4644 (95.16%)
- Refused: 2 (0.04%)
- Don't know: 31 (0.64%)

Column: MCQ160F
- Yes: 190 (3.89%)
- No: 4687 (96.05%)
- Don't know: 3 (0.06%)

Column: MCQ160K
- Yes: 299 (6.13%)
- No: 4573 (93.71%)
- Don't know: 8 (0.16%)

Column: MCQ160L
- Yes: 153 (3.14%)
- No: 4715 (96.62%)
- Don't know: 12 (0.25%)

Column: MCQ220
- Yes: 386 (7.91%)
- No: 4489 (91.99%)
- Don't know: 5 (0.10%)


In [54]:
print_categorical_info(med_cond_01_02_selected, "Medical Conditions 2001-2002")


Categorical Column Analysis for Dataset: Medical Conditions 2001-2002

Column: MCQ160B
- Yes: 187 (3.46%)
- No: 5200 (96.12%)
- Refused: 1 (0.02%)
- Don't know: 22 (0.41%)

Column: MCQ160C
- Yes: 240 (4.44%)
- No: 5131 (94.84%)
- Refused: 1 (0.02%)
- Don't know: 38 (0.70%)

Column: MCQ160F
- Yes: 204 (3.77%)
- No: 5197 (96.06%)
- Don't know: 9 (0.17%)

Column: MCQ160K
- Yes: 291 (5.38%)
- No: 5103 (94.33%)
- Don't know: 16 (0.30%)

Column: MCQ160L
- Yes: 158 (2.92%)
- No: 5241 (96.88%)
- Don't know: 11 (0.20%)

Column: MCQ220
- Yes: 520 (9.61%)
- No: 4883 (90.26%)
- Don't know: 7 (0.13%)


In [55]:
print_categorical_info(diabetes_99_00_selected, "Diabetes 1999-2000")


Categorical Column Analysis for Dataset: Diabetes 1999-2000

Column: DIQ010
- Yes: 489 (5.15%)
- No: 8936 (94.18%)
- Borderline: 58 (0.61%)
- Refused: 1 (0.01%)
- Don't know: 4 (0.04%)


In [56]:
print_categorical_info(diabetes_01_02_selected, "Diabetes 2001-2002")


Categorical Column Analysis for Dataset: Diabetes 2001-2002

Column: DIQ010
- Yes: 528 (5.04%)
- No: 9861 (94.20%)
- Borderline: 78 (0.75%)
- Don't know: 1 (0.01%)


In [57]:
print_categorical_info(blood_pressure_99_00_selected, "High Blood Pressure 1999-2000")


Categorical Column Analysis for Dataset: High Blood Pressure 1999-2000

Column: BPQ020
- Yes: 1532 (25.74%)
- No: 4408 (74.07%)
- Don't know: 11 (0.18%)


In [58]:
print_categorical_info(blood_pressure_01_02_selected, "High Blood Pressure 2001-2002")


Categorical Column Analysis for Dataset: High Blood Pressure 2001-2002

Column: BPQ020
- Yes: 1681 (25.73%)
- No: 4843 (74.12%)
- Don't know: 10 (0.15%)


In [59]:
print_categorical_info(physical_99_00_selected, "Physical Activities 1999-2000")


Categorical Column Analysis for Dataset: Physical Activities 1999-2000

Column: PAQ500
- more active: 1316 (18.28%)
- less active, or: 1406 (19.53%)
- About the same?: 4467 (62.06%)
- Refused: 6 (0.08%)
- Don't know: 3 (0.04%)


In [60]:
print_categorical_info(physical_01_02_selected, "Physical Activities 2001-2002")


Categorical Column Analysis for Dataset: Physical Activities 2001-2002

Column: PAQ500
- more active: 1399 (17.97%)
- less active, or: 1598 (20.53%)
- About the same?: 4779 (61.40%)
- Refused: 4 (0.05%)
- Don't know: 4 (0.05%)


In [61]:
print_categorical_info(eden_99_00_selected, "Edentulous 1999-2000")


Categorical Column Analysis for Dataset: Edentulous 1999-2000

Column: OHXEDEN
- Yes: 418 (95.87%)
- No: 18 (4.13%)


In [62]:
print_categorical_info(eden_01_02_selected, "Edentulous 2001-2002")


Categorical Column Analysis for Dataset: Edentulous 2001-2002

Column: OHXEDEN
- Yes: 425 (99.77%)
- No: 1 (0.23%)


In [63]:
print_categorical_info(oral_99_00_selected, "Oral Health 1999-2000")


Categorical Column Analysis for Dataset: Oral Health 1999-2000

Column: OHQ080
- Yes: 147 (7.88%)
- No: 1718 (92.07%)
- Don't know: 1 (0.05%)

Column: OHQ100
- Yes: 128 (6.86%)
- No: 1738 (93.14%)

Column: OHQ110
- Yes: 95 (5.09%)
- No: 1765 (94.59%)
- Don't know: 6 (0.32%)

Column: OHQ020
- Always,: 108 (1.98%)
- Very often,: 97 (1.78%)
- Often,: 146 (2.68%)
- Sometimes,: 535 (9.83%)
- Seldom, or: 552 (10.14%)
- Never?: 3998 (73.45%)
- Refused: 5 (0.09%)
- Don't know: 2 (0.04%)

Column: OHQ040
- Yes: 1163 (49.87%)
- No: 1155 (49.53%)
- Refused: 5 (0.21%)
- Don't know: 9 (0.39%)

Column: OHQ050
- 2 or more times a year: 2685 (46.51%)
- Once a year: 1684 (29.17%)
- Less than once a year: 502 (8.70%)
- Whenever needed, no regular schedule: 894 (15.49%)
- Refused: 1 (0.02%)
- Don't know: 7 (0.12%)


In [64]:
print_categorical_info(oral_01_02_selected, "Oral Health 2001-2002")


Categorical Column Analysis for Dataset: Oral Health 2001-2002

Column: OHQ085
- Yes: 413 (7.11%)
- No: 5394 (92.84%)
- Refused: 3 (0.05%)

Column: OHQ105
- Yes: 270 (4.65%)
- No: 5533 (95.23%)
- Refused: 3 (0.05%)
- Don't know: 4 (0.07%)

Column: OHQ115
- Yes: 278 (4.78%)
- No: 5517 (94.96%)
- Refused: 3 (0.05%)
- Don't know: 12 (0.21%)

Column: OHQ020
- Always,: 91 (1.52%)
- Very often,: 124 (2.07%)
- Often,: 133 (2.22%)
- Sometimes,: 518 (8.65%)
- Seldom, or: 624 (10.42%)
- Never?: 4490 (74.97%)
- Refused: 4 (0.07%)
- Don't know: 5 (0.08%)

Column: OHQ040
- Yes: 1419 (54.91%)
- No: 1155 (44.70%)
- Refused: 3 (0.12%)
- Don't know: 7 (0.27%)

Column: OHQ050
- 2 or more times a year: 3286 (49.89%)
- Once a year: 1786 (27.12%)
- Less than once a year: 479 (7.27%)
- Whenever needed, no regular schedule: 1029 (15.62%)
- Don't know: 6 (0.09%)


### Missing Values and % for each dataset

In [66]:
# Function to calculate missing values

missing_values <- function(df) {
  missing_count <- colSums(is.na(df))
  missing_percent <- (missing_count/nrow(df)) * 100

  missing_df <- data.frame(
    variable = names(missing_count),
    n_missing = missing_count,
    percent_missing = round(missing_percent, 2)
  ) %>%
    arrange(desc(n_missing))

  return(missing_df)
}

In [67]:
demo_99_00_missing <- missing_values(demo_99_00_selected)
print(demo_99_00_missing)

         variable n_missing percent_missing
DMDEDUC2 DMDEDUC2      5087           51.05
INDFMPIR INDFMPIR      1483           14.88
SEQN         SEQN         0            0.00
RIAGENDR RIAGENDR         0            0.00
RIDAGEYR RIDAGEYR         0            0.00
RIDRETH1 RIDRETH1         0            0.00


In [68]:
demo_01_02_missing <- missing_values(demo_01_02_selected)
print(demo_01_02_missing)

         variable n_missing percent_missing
DMDEDUC2 DMDEDUC2      5632           51.02
INDFMPIR INDFMPIR       794            7.19
SEQN         SEQN         0            0.00
RIAGENDR RIAGENDR         0            0.00
RIDAGEYR RIDAGEYR         0            0.00
RIDRETH1 RIDRETH1         0            0.00


In [69]:
smoking_99_00_missing <- missing_values(smoking_99_00_selected)
print(smoking_99_00_missing)

       variable n_missing percent_missing
SMQ020   SMQ020         3            0.06
SEQN       SEQN         0            0.00


In [70]:
smoking_01_02_missing <- missing_values(smoking_01_02_selected)
print(smoking_01_02_missing)

       variable n_missing percent_missing
SMQ020   SMQ020         4            0.07
SEQN       SEQN         0            0.00


In [71]:
med_cond_99_00_missing <- missing_values(med_cond_99_00_selected)
print(med_cond_99_00_missing)

        variable n_missing percent_missing
MCQ160B  MCQ160B      4613           48.59
MCQ160C  MCQ160C      4613           48.59
MCQ160F  MCQ160F      4613           48.59
MCQ160K  MCQ160K      4613           48.59
MCQ160L  MCQ160L      4613           48.59
MCQ220    MCQ220      4613           48.59
SEQN        SEQN         0            0.00


In [72]:
med_cond_01_02_missing <- missing_values(med_cond_01_02_selected)
print(med_cond_01_02_missing)

        variable n_missing percent_missing
MCQ160B  MCQ160B      5060           48.33
MCQ160C  MCQ160C      5060           48.33
MCQ160F  MCQ160F      5060           48.33
MCQ160K  MCQ160K      5060           48.33
MCQ160L  MCQ160L      5060           48.33
MCQ220    MCQ220      5060           48.33
SEQN        SEQN         0            0.00


In [73]:
diabetes_99_00_missing <- missing_values(diabetes_99_00_selected)
print(diabetes_99_00_missing)

       variable n_missing percent_missing
DIQ010   DIQ010         5            0.05
SEQN       SEQN         0            0.00


In [74]:
diabetes_01_02_missing <- missing_values(diabetes_01_02_selected)
print(diabetes_01_02_missing)

       variable n_missing percent_missing
DIQ010   DIQ010         2            0.02
SEQN       SEQN         0            0.00


In [75]:
blood_pressure_99_00_missing <- missing_values(blood_pressure_99_00_selected)
print(blood_pressure_99_00_missing)

       variable n_missing percent_missing
BPQ020   BPQ020        93            1.54
SEQN       SEQN         0            0.00


In [76]:
blood_pressure_01_02_missing <- missing_values(blood_pressure_01_02_selected)
print(blood_pressure_01_02_missing)

       variable n_missing percent_missing
BPQ020   BPQ020       100            1.51
SEQN       SEQN         0            0.00


In [77]:
physical_99_00_missing <- missing_values(physical_99_00_selected)
print(physical_99_00_missing)

       variable n_missing percent_missing
PAQ500   PAQ500      1990           21.66
SEQN       SEQN         0            0.00


In [78]:
physical_01_02_missing <- missing_values(physical_01_02_selected)
print(physical_01_02_missing)

       variable n_missing percent_missing
PAQ500   PAQ500      2310           22.88
SEQN       SEQN         0            0.00


In [79]:
eden_99_00_missing <- missing_values(eden_99_00_selected)
print(eden_99_00_missing)

        variable n_missing percent_missing
OHXEDEN  OHXEDEN      8151           94.92
SEQN        SEQN         0            0.00


In [80]:
eden_01_02_missing <- missing_values(eden_01_02_selected)
print(eden_01_02_missing)

        variable n_missing percent_missing
OHXEDEN  OHXEDEN      9172           95.56
SEQN        SEQN         0            0.00


In [81]:
oral_99_00_missing <- missing_values(oral_99_00_selected)
print(oral_99_00_missing)

       variable n_missing percent_missing
OHQ080   OHQ080      7364           79.78
OHQ100   OHQ100      7364           79.78
OHQ110   OHQ110      7364           79.78
OHQ040   OHQ040      6898           74.73
OHQ020   OHQ020      3787           41.03
OHQ050   OHQ050      3457           37.45
SEQN       SEQN         0            0.00


In [82]:
oral_01_02_missing <- missing_values(oral_01_02_selected)
print(oral_01_02_missing)

       variable n_missing percent_missing
OHQ040   OHQ040      7543           74.48
OHQ085   OHQ085      4317           42.63
OHQ105   OHQ105      4317           42.63
OHQ115   OHQ115      4317           42.63
OHQ020   OHQ020      4138           40.86
OHQ050   OHQ050      3541           34.97
SEQN       SEQN         0            0.00


### Combine Demographics datasets and filter with age >= 60

In [ ]:
combined_demo <- bind_rows(demo_99_00_selected, demo_01_02_selected)

In [119]:
dim(combined_demo)

[1] 21004     6

In [120]:
elderly_demo_data <- combined_demo %>%
  filter(RIDAGEYR >= 60)

In [123]:
dim(elderly_demo_data)

[1] 3706    6

### Merge datasets in a one and complete data frame 

In [83]:
datasets_99_00 <- list(
  demo_99_00_selected, smoking_99_00_selected, med_cond_99_00_selected,
  diabetes_99_00_selected, blood_pressure_99_00_selected, physical_99_00_selected,
  eden_99_00_selected, oral_99_00_selected
)

datasets_01_02 <- list(
  demo_01_02_selected, smoking_01_02_selected, med_cond_01_02_selected,
  diabetes_01_02_selected, blood_pressure_01_02_selected, physical_01_02_selected,
  eden_01_02_selected, oral_01_02_selected
)

# Horizontal union for period 1999/00 and 2001/02

df_99_00 <- Reduce(function(x, y) full_join(x, y, by = "SEQN"), datasets_99_00)

df_01_02 <- Reduce(function(x, y) full_join(x, y, by = "SEQN"), datasets_01_02)

In [88]:
# Vertical union for these two periods

df_final <- bind_rows(df_99_00, df_01_02)

dim(df_final)

[1] 21004    26

In [91]:
head(df_final)

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,DMDEDUC2,INDFMPIR,SMQ020,MCQ160B,MCQ160C,MCQ160F,...,OHXEDEN,OHQ080,OHQ100,OHQ110,OHQ020,OHQ040,OHQ050,OHQ085,OHQ105,OHQ115
,<dbl>,<fct>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,...,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1,Female,2,Non-Hispanic Black,NA,0.86,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,Male,77,Non-Hispanic White,College Graduate or above,5.00,No,No,No,No,...,NA,No,No,No,Never?,NA,2 or more times a year,NA,NA,NA
3,3,Female,10,Non-Hispanic White,NA,1.47,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,Once a year,NA,NA,NA
4,4,Male,1,Non-Hispanic Black,NA,0.57,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,5,Male,49,Non-Hispanic White,College Graduate or above,5.00,Yes,No,No,No,...,NA,No,No,No,Never?,Yes,2 or more times a year,NA,NA,NA
6,6,Female,19,Other Race - Including Multi-Racial,NA,1.21,NA,NA,NA,NA,...,NA,NA,NA,NA,Never?,NA,Once a year,NA,NA,NA


In [ ]:
# Condition to exclude people younger than 60 years old

elderly_df_final <- df_final %>%
  filter(RIDAGEYR >= 60)

dim(elderly_df_final)

[1] 3706   26

In [90]:
# Remove every row with a least one NA values

elderly_df_final_no_na <- na.omit(elderly_df_final)

dim(elderly_df_final_no_na)

[1]  0 26

### Next steps

In [ ]:
# Feature Engineering

process_features <- function(data) {
    processed_data <- data %>%
        mutate(
            gender = factor(x = RIAGENDR, levels = c(1, 2), labels = c("Male", "Female"))
        ) %>%
        mutate(
            ethnicity = case_when(
                RIDRETH1 == 1 ~ "Mexican American",
                RIDRETH1 == 2 ~ "Other Hispanic",
                RIDRETH1 == 3 ~ "Non-Hispanic White",
                RIDRETH1 == 4 ~ "Non-Hispanic Black",
                RIDRETH1 == 5 ~ "Other Race - Including Multi-Racial",
                TRUE ~ NA_character_
            ),
            ethnicity_num = as.numeric(RIDRETH1)
        ) %>%
        mutate(
            education = case_when(
                DMDEDUC2 == 1 ~ "Less than 9th grade",
                DMDEDUC2 == 2 ~ "9-11th grade",
                DMDEDUC2 == 3 ~ "High school graduate/GED",
                DMDEDUC2 == 4 ~ "Some college or AA degree",
                DMDEDUC2 == 5 ~ "College graduate or above",
                DMDEDUC2 %in% c(7, 9) ~ NA_character_
            ),
            education_3cat = case_when(
                DMDEDUC2 %in% c(1, 2) ~ "Less than 12 years",
                DMDEDUC2 == 3 ~ "12 years",
                DMDEDUC2 %in% c(4, 5) ~ "More than 12 years",
                TRUE ~ NA_character_
            ),
            education_years = case_when(
                DMDEDUC2 == 1 ~ 8, # Less than 9th grade
                DMDEDUC2 == 2 ~ 10, # 9-11th grade
                DMDEDUC2 == 3 ~ 12, # High school/GED
                DMDEDUC2 == 4 ~ 14, # Some college/AA
                DMDEDUC2 == 5 ~ 16, # College graduate
                TRUE ~ NA_real_
            )
        ) %>%
        # Binary indicators for each ethnic category
        mutate(
            is_mexican_american = RIDRETH1 == 1,
            is_other_hispanic = RIDRETH1 == 2,
            is_nh_white = RIDRETH1 == 3,
            is_nh_black = RIDRETH1 == 4,
            is_other_race = RIDRETH1 == 5
        ) %>%
        # Binary indicators for each educational level
        mutate(
            is_less_than_hs = DMDEDUC2 %in% c(1, 2),
            is_hs_grad = DMDEDUC2 == 3,
            is_some_college = DMDEDUC2 == 4,
            is_college_grad = DMDEDUC2 == 5
        ) %>%
        mutate(
            income_category = case_when(
                INDHHIN2 %in% c(1:4) ~ "Under $20,000",
                INDHHIN2 %in% c(5:7) ~ "$20,000 to $44,999",
                INDHHIN2 %in% c(8:10) ~ "$45,000 to $74,999",
                INDHHIN2 == 13 ~ "$75,000 to $99,999",
                INDHHIN2 == 14 ~ "$100,000 and Over",
                INDHHIN2 %in% c(77, 99) ~ NA_character_,
                TRUE ~ NA_character_
            )
        ) %>%
        mutate(
            income_numeric = case_when(
                INDHHIN2 == 1 ~ 2500, # $0-$4,999
                INDHHIN2 == 2 ~ 7500, # $5,000-$9,999
                INDHHIN2 == 3 ~ 12500, # $10,000-$14,999
                INDHHIN2 == 4 ~ 17500, # $15,000-$19,999
                INDHHIN2 == 5 ~ 22500, # $20,000-$24,999
                INDHHIN2 == 6 ~ 30000, # $25,000-$34,999
                INDHHIN2 == 7 ~ 40000, # $35,000-$44,999
                INDHHIN2 == 8 ~ 50000, # $45,000-$54,999
                INDHHIN2 == 9 ~ 60000, # $55,000-$64,999
                INDHHIN2 == 10 ~ 70000, # $65,000-$74,999
                INDHHIN2 == 13 ~ 87500, # $75,000-$99,999
                INDHHIN2 == 14 ~ 125000, # $100,000 and Over
                TRUE ~ NA_real_
            )
        ) %>%
        mutate(
            permanent_teeth = rowSums(select(., starts_with("OHX") & ends_with("TC")) == 1, na.rm = TRUE),
            missing_teeth = rowSums(select(., starts_with("OHX") & ends_with("TC")) == 2, na.rm = TRUE),
            root_fragments = rowSums(select(., starts_with("OHX") & ends_with("TC")) == 3, na.rm = TRUE),
            perc_permanent = (permanent_teeth / (permanent_teeth + missing_teeth + root_fragments)) * 100,
            dental_status = case_when(
                permanent_teeth >= 20 ~ "Adequate dentition",
                permanent_teeth >= 10 ~ "Partial dentition",
                TRUE ~ "Poor dentition"
            )
        ) %>%
        mutate(
            periodontal_disease = case_when(
                # Definition CDC/AAP as in the paper (this is a simplified example)
                rowMeans(select(., ends_with("CSC")), na.rm = TRUE) >= 4 ~ "Moderate/Severe",
                TRUE ~ "No/Mild"
            )
        )

    return(processed_data)
}


In [ ]:
preprocessed_data <- process_features(features_selected)
head(preprocessed_data)


## nhanesA packages functionalities

### Create dataframe using NHANES codes

In [ ]:
DF <- nhanesA::nhanes(nh_table = "OHQ")
DF$OHQ080


In [ ]:
nhanesA::nhanesCodebook("OHQ", "OHQ080")


In [ ]:
head(DF[, c("OHQ020", "OHQ040", "OHQ050", "OHQ080", "OHQ100", "OHQ110")])


### How to drop all NA values

In [ ]:
DF_filtered <- na.omit(DF[, c("OHQ020", "OHQ040", "OHQ050", "OHQ080", "OHQ100", "OHQ110")])
dim(DF_filtered)


### Explore Nhanes attributes

In [ ]:
attributes_df <- nhanesAttr(nh_table = "OHQ")
attributes_df


In [ ]:
attributes_df <- nhanesTableSummary(nh_table = "OHQ")
attributes_df


### Display codebook for selected variable

In [ ]:
nhanesCodebook(nh_table = "OHQ")$OHQ100


In [ ]:
# You can do the same with nhanesCodebookFromURL: Download and parse an NHANES doc file from a URL

nhanesCodebookFromURL(url = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/OHQ_G.htm")$OHQ885


In [ ]:
dxa_c_s <- nhanesDXA(2003, suppl = TRUE)
head(dxa_c_s)


### Download and parse NHANES manifests

In [ ]:
manifest <- nhanesManifest(sizes = FALSE)
dim(manifest)


In [ ]:
varmf <- nhanesManifest("variables", component = "lab")
head(varmf)


### Perform a search over the comprehensive NHANES variable list

In [ ]:
length(bladder)


In [ ]:
head(bladder)


### Explore Data Groups such as Questionnarie, Demographics...

In [ ]:
nhanesA::nhanesTables(data_group = "QUESTIONNAIRE", year = 2000)


In [ ]:
exam <- nhanesA::nhanesTables("QUESTIONNAIRE", 2000)
dim(exam)


In [ ]:
nhanesA::nhanesTableSummary("DEMO_D", use = "codebook")


In [ ]:
# Displays a list of variables in the specified NHANES table.
exam_ohx <- nhanesTableVars("EXAM", "OHX_E", details = TRUE, nchar = 50)
dim(exam_ohx)


In [ ]:
head(exam_ohx)


In [ ]:
# Display code translation information.

nhanesTranslate("DEMO_B", c("DMDBORN", "DMDCITZN"))
